In [1]:
# Imports
import pandas as pd
import time
import requests
import re

from bs4 import BeautifulSoup
#from lxml import html

In [2]:
# Setting base url
url = 'https://www.sailorsongs.com/'

In [3]:
# Requesting links page
res_links = requests.get(f'{url}lyrics.html', headers={"User-Agent": '42.0.2311.135'})
res_links.status_code

200

In [4]:
# Converting requests result into list of links
links_soup = BeautifulSoup(res_links.content, 'lxml')
links_list = links_soup.find_all('a', href=True)
links_list = [x['href'] for x in links_list]

In [5]:
# Dropping non-song links from list
del links_list[:7]
del links_list[-6:]

In [6]:
# Creating empty dictionary to hold songs
shanties_dict = {}

In [7]:
# Iterating through
for link in links_list:
    # Requesting lyrics page
    res_lyrics = requests.get(f'{url}{link}')
    
    # Putting in a small sleep to limit requests on server to under 100 per minutes
    time.sleep(.75)
    # Uncomment to check status of scrape. print(f'{res_lyrics.status_code} {link}')
    
    # Pulling song title
    title = BeautifulSoup(res_lyrics.text).title.text

    # Converting lyrics request into list
    lyrics_soup = BeautifulSoup(res_lyrics.content, 'lxml')
    lyrics_raw = lyrics_soup.find_all('p')
    # Checking to see if lyrics block has intro text
    if lyrics_raw[0].text.find('www.SailorSongs.com') != -1:
        del lyrics_raw[0]
    # Deleting non-lyrics from list
    del lyrics_raw[-1]

    # Converting soup into list
    lyrics_raw = [x.text for x in lyrics_raw]
    
    # Converting list of strings to single string
    lyrics = ''
    lyrics = lyrics.join(lyrics_raw)
    
    # Updating Shanties Dict
    shanties_dict.update({title : lyrics})

In [8]:
# Creating data frame for songs and lyrics
sailor_songs_songs = pd.DataFrame(shanties_dict.items(), columns=['title', 'lyrics'])

In [9]:
# Writing to csv for use in other scripts
sailor_songs_songs.to_csv('../../data/sailor_songs_songs.csv')